In [1]:
import sys
sys.path.append('/users/gxb18167/EEG-NLP')

from src import data
from src import utils
from src import Networks

import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
C:\Users\gxb18167\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\train_NER.pkl"

test_path = r"C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\Named-Entity-Classification\Data-Management\test_NER.pkl"

In [3]:
d = data.Data()
util = utils.Utils()

In [4]:
train_NE, train_EEG_segments, train_Classes = d.NER_read_custom_files(train_path)

In [5]:
train_NE[0]

['Henry', 'Ford']

In [6]:
vector_size = 50
#parameters['vector_size'] = vector_size
window = 5
#parameters['window'] = window
min_count = 1
#parameters['min_count'] = min_count
workers = 4

In [7]:
train_word_embeddings, train_NE_embedded = util.NER_Word2Vec(train_NE, vector_size, window, min_count, workers)

In [8]:
len(train_NE_embedded)

585

In [9]:
train_NE_expanded = util.NER_expanded_NER_list(train_EEG_segments, train_NE_embedded)

In [16]:
train_NE_expanded[0][0].shape

(50,)

In [11]:
max_seq_len = max([len(i) for i in train_NE_expanded])
#padding function
for i in range(len(train_NE_expanded)):
    padding_count = max_seq_len - len(train_NE_expanded[i])
    for j in range(padding_count):
        train_NE_expanded[i].append(np.zeros(vector_size))





In [12]:
train_NE_expanded = np.array(train_NE_expanded)

In [13]:
train_NE_expanded.shape

(21875, 7, 50)

In [17]:
import torch
from transformers import BertTokenizer, BertModel

class NER_BERT:
    def __init__(self, model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def NER_BERT(self, Word_Labels_List):
        embedded_input = []
        for named_entity in Word_Labels_List:
            inputs = self.tokenizer(named_entity, return_tensors="pt", padding=True, truncation=True)
            outputs = self.model(**inputs)
            last_hidden_states = outputs.last_hidden_state
            embedded_input.append(last_hidden_states.mean(dim=1).squeeze().detach().numpy())

        return embedded_input

In [18]:
ner_bert = NER_BERT()

C:\Users\gxb18167\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\gxb18167\Anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gxb18167\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [19]:
embeddings = ner_bert.NER_BERT(['hello', 'world'])